<a href="https://colab.research.google.com/github/SyaoranClone/Neural-Probabilistic-Language-Model-/blob/master/Neural_Probabilistic_Language_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import gensim as gs
import nltk
import numpy as np
import time
import sys
import random
import collections
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
nltk.download('gutenberg')
nltk.download('punkt')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from nltk.corpus import gutenberg

In [0]:
data_words  = gutenberg.words('shakespeare-macbeth.txt')
data_sents  = gutenberg.sents('shakespeare-macbeth.txt')

In [0]:
def build_dictionaries(words):
  count = collections.Counter(words).most_common()
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary) #len(dictionary) increases each iteration
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return dictionary, reverse_dictionary
 
dictionary, reverse_dictionary = build_dictionaries(data_words)

In [0]:
vocab_size = len(dictionary)
embedding_size = 100

In [0]:
def prepare_data(data,max_len,step=1):
  sections = []
  section_labels = []
  for i in range(0,len(data)-max_len,step):
    sections.append(data[i:i+max_len])
    section_labels.append([data[i+max_len]])
    
  train_x = np.zeros((len(sections),max_len,1))
  train_y = np.zeros((len(sections),vocab_size))
  
  for i,section in enumerate(sections):
    for j,word in enumerate(section):
      train_x[i,j] = dictionary[word]
    train_y[i,dictionary[section_labels[i][0]]] = 1.0
  p_train_x = []
  for i,_ in enumerate(train_x):
    p_train_x.append(np.reshape(np.array(train_x[i]), [-1, max_len])[0])
  return p_train_x,train_y

In [0]:
class LSTM:
  def __init__(self,input_size,embbeding_size,hidden_nodes,learning_rate):
    self.lr = learning_rate
    self.eb_size = embbeding_size
    self.x = tf.placeholder(tf.int32, [None ,input_size])
    #self.ex = tf.placeholder(tf.float32,[None,vocab_size])
    self.y = tf.placeholder(tf.float32,[None,vocab_size])
    
    #w1 b1 embedding layer
#     self.w1 = tf.Variable(tf.random_normal([vocab_size, self.eb_size]))
#     self.b1 = tf.Variable(tf.random_normal([self.eb_size]))
#     self.hidden_representation = tf.add(tf.matmul(self.ex,self.w1), self.b1)
#     inputs = tf.nn.embedding_lookup(self.hidden_representation,self.x)

    embedding = tf.Variable(tf.random_uniform((vocab_size, self.eb_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, self.x) # use tf.nn.embedding_lookup to get the hidden layer
    
    self.w = tf.Variable(tf.random_normal([hidden_nodes,vocab_size]))
    self.b = tf.Variable(tf.random_normal([vocab_size]))
    
    #build RNN layer
    with tf.name_scope("RNN_Layers"):
      cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(hidden_nodes,reuse=tf.AUTO_REUSE),tf.contrib.rnn.BasicLSTMCell(hidden_nodes,reuse=tf.AUTO_REUSE)]) 
      #tf.contrib.rnn.BasicLSTMCell(hidden_nodes,forget_bias=1.0,reuse=tf.AUTO_REUSE)
      output,state = tf.contrib.rnn.static_rnn(cell,tf.unstack(embed,input_size,1),dtype=tf.float32)
    
    self.logits = tf.matmul(output[-1],self.w)+self.b
    self.prediction = tf.nn.softmax(self.logits)
    self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits,labels=self.y))
    self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=self.lr)
    self.train_op = self.optimizer.minimize(self.loss)
    
    with tf.name_scope("Accuracy"):
      correct_pred = tf.equal(tf.argmax(self.prediction, 1), tf.argmax(self.y, 1))
      accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
  
  #This function is used to get a random word from the prediction according to its probabilities.
  def sample(self,preds,temperature=1.0):
    if temperature <= 0:
      return np.argmax(preds)
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    print(probas)
    return np.argmax(probas)
    
  def generate_text_from_trained_data(self,data,prediction_length,sess):
    start_index = random.randint(0, len(data) - max_len - 1)
    test_start = data[start_index: start_index + max_len]
    result = test_start
    X_test = [[dictionary[word] for word in test_start]]
#     X_test = np.zeros((1,max_len,self.eb_size))
#     for i,word in enumerate(test_start):
#       X_test[0,i] = word_model[word]
      
    print(" Result:")
    for _ in range(prediction_length): 
      pred = sess.run(self.prediction,feed_dict={self.x:X_test})
      #pred = pred.reshape(-1)
      pred = int(tf.argmax(pred,1).eval())
      #print(int(tf.argmax(pred,1).eval()))
      #pred = self.sample(pred,temperature=0.7)
      #print(pred)
      #next_word = word_model.wv.most_similar(positive=[pred])[0][0]
      next_word = reverse_dictionary[pred]
      #print(next_word)
      test_start.append(next_word)
      result.append(next_word)
      
      test_start = test_start[-max_len:]
      X_test = [[dictionary[word] for word in test_start]]
#       X_test = np.zeros((1,max_len,self.eb_size))
#       for j,word in enumerate(test_start):
#         X_test[0,j] = word_model[word]
    print(*result)

        

  def next_batch(self, data, batch_start, batch_size):
        len_data = len(data)
        batch_end = min(batch_start + batch_size, len_data)
        return data[batch_start:batch_end]
      
  def train(self,splited_data,x_data,y_data,num_eponches=1,batch_size=1,prediction_length=50,checkpoint_path="/content/gdrive/My Drive/Colab Notebooks/Trained Models/"):
    saver = tf.train.Saver()
    with tf.Session() as session:
      len_data = len(x_data)
      if tf.train.latest_checkpoint(checkpoint_path):
        print("Restore checkpoint.")
        saver = tf.train.Saver()
        saver.restore(session, tf.train.latest_checkpoint(checkpoint_path))
      else:
        session.run(tf.global_variables_initializer())
        print("No checkpoint found! start a new training...")
      for epoch in range(1, num_eponches + 1):
                epoch_start_time = time.time()
                for batch_start in range(0, len_data, batch_size):
                    optimiser_value, loss_value = session.run([self.train_op,self.loss], feed_dict={
                        self.x: self.next_batch(x_data, batch_start, batch_size),
                        self.y: self.next_batch(y_data, batch_start, batch_size)})
                    sys.stdout.write(
                        "\rEpoch: {}/{}, Batch: {}/{}, Training loss: {}".format(
                            epoch,
                            num_eponches,
                            (batch_start / batch_size) + 1,
                            len_data / batch_size,
                            np.mean(loss_value)
                        )
                    )
                    sys.stdout.flush()
                    
                    #self.generate_text_from_trained_data(splited_data,prediction_length,session)
                print (" Time: {} s".format(time.time() - epoch_start_time))
                model_name = "cp_language_model.ckpt"
                saver.save(session, checkpoint_path + model_name)
                self.generate_text_from_trained_data(splited_data,prediction_length,session)
                
    

In [0]:
class biLSTM:
  def __init__(self,input_size,embbeding_size,hidden_nodes,learning_rate):
    self.lr = learning_rate
    self.eb_size = embbeding_size
    self.x = tf.placeholder(tf.int32, [None ,input_size])
    self.y = tf.placeholder(tf.float32,[None,vocab_size])
    
    embedding = tf.Variable(tf.random_uniform((vocab_size, self.eb_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, self.x) # use tf.nn.embedding_lookup to get the hidden layer
    
    self.w = tf.Variable(tf.random_normal([2*hidden_nodes,vocab_size])) # Hidden layer weights => 2*n_hidden because of forward + backward cells
    self.b = tf.Variable(tf.random_normal([vocab_size]))
    
    #build RNN layer
    with tf.name_scope("RNN_Layers"):
      # Forward direction cell
      lstm_fw_cell = tf.contrib.rnn.BasicLSTMCell(hidden_nodes, forget_bias=1.0,reuse=tf.AUTO_REUSE)
      # Backward direction cell
      lstm_bw_cell = tf.contrib.rnn.BasicLSTMCell(hidden_nodes, forget_bias=1.0,reuse=tf.AUTO_REUSE)
      output, _, _ = tf.contrib.rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell,tf.unstack(embed,input_size,1),dtype=tf.float32)
      
    
    self.logits = tf.matmul(output[-1],self.w)+self.b
    self.prediction = tf.nn.softmax(self.logits)
    self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits,labels=self.y))
    self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=self.lr)
    self.train_op = self.optimizer.minimize(self.loss)
    
    with tf.name_scope("Accuracy"):
      correct_pred = tf.equal(tf.argmax(self.prediction, 1), tf.argmax(self.y, 1))
      accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
  
  #This function is used to get a random word from the prediction according to its probabilities.
  def sample(self,preds,temperature=1.0):
    if temperature <= 0:
      return np.argmax(preds)
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    print(probas)
    return np.argmax(probas)
    
  def generate_text_from_trained_data(self,data,prediction_length,sess):
    start_index = random.randint(0, len(data) - max_len - 1)
    test_start = data[start_index: start_index + max_len]
    result = test_start
    X_test = [[dictionary[word] for word in test_start]]
      
    print(" Result:")
    for _ in range(prediction_length): 
      pred = sess.run(self.prediction,feed_dict={self.x:X_test})
      pred = int(tf.argmax(pred,1).eval())
      next_word = reverse_dictionary[pred]
      test_start.append(next_word)
      result.append(next_word)
      
      test_start = test_start[-max_len:]
      X_test = [[dictionary[word] for word in test_start]]
    print(*result)

        

  def next_batch(self, data, batch_start, batch_size):
        len_data = len(data)
        batch_end = min(batch_start + batch_size, len_data)
        return data[batch_start:batch_end]
      
  def train(self,splited_data,x_data,y_data,num_eponches=1,batch_size=1,prediction_length=50,checkpoint_path="/content/gdrive/My Drive/Colab Notebooks/Trained Models/"):
    saver = tf.train.Saver()
    with tf.Session() as session:
      len_data = len(x_data)
#       if tf.train.latest_checkpoint(checkpoint_path):
#         print("Restore checkpoint.")
#         saver = tf.train.Saver()
#         saver.restore(session, tf.train.latest_checkpoint(checkpoint_path))
#       else:
      session.run(tf.global_variables_initializer())
       #print("No checkpoint found! start a new training...")
      for epoch in range(1, num_eponches + 1):
                epoch_start_time = time.time()
                for batch_start in range(0, len_data, batch_size):
                    optimiser_value, loss_value = session.run([self.train_op,self.loss], feed_dict={
                        self.x: self.next_batch(x_data, batch_start, batch_size),
                        self.y: self.next_batch(y_data, batch_start, batch_size)})
                    sys.stdout.write(
                        "\rEpoch: {}/{}, Batch: {}/{}, Training loss: {}, Perplexity: {}".format(
                            epoch,
                            num_eponches,
                            (batch_start / batch_size) + 1,
                            len_data / batch_size,
                            np.mean(loss_value),
                            tf.exp(loss_value).eval()
                        )
                    )
                    sys.stdout.flush()
                    
                    #self.generate_text_from_trained_data(splited_data,prediction_length,session)
                print (" Time: {} s".format(time.time() - epoch_start_time))
                self.generate_text_from_trained_data(splited_data,prediction_length,session)
      model_name = "biLSTM_language_model.ckpt"
      saver.save(session, checkpoint_path + model_name)

In [0]:
max_len = 10
hidden_nodes=1024
learning_rate =0.01
total_epoches = 500
batch_size = 180

data_split_by_word = gutenberg.words('shakespeare-macbeth.txt')
train_x,train_y = prepare_data(data_split_by_word,10,1)


In [0]:
x_integers = [[dictionary[str(data_split_by_word[i])] for i in range(0, 10)]]
#x_integers = np.reshape(np.array(x_integers), [-1, 10])[0]
x_integers
np.array(x_integers)

array([[ 574,   25, 1517,    7,   44,   83, 1518, 1519, 1520,  575]])

In [0]:
lstm_model = LSTM(max_len,embedding_size,hidden_nodes,learning_rate)

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Use tf.cast instead.


In [0]:
lstm_model.train(data_split_by_word,train_x,train_y,total_epoches,batch_size,10)

No checkpoint found! start a new training...
Epoch: 1/100, Batch: 129.0/128.5, Training loss: 6.878818035125732 Time: 7.447726726531982 s
 Result:
beene , my sences would haue cool ' d To , , , , and and , , and and ,
Epoch: 2/100, Batch: 129.0/128.5, Training loss: 6.5784173011779785 Time: 7.08791971206665 s
 Result:
which I see before me , The Handle toward my , , and and , , and and , , ,
Epoch: 3/100, Batch: 129.0/128.5, Training loss: 6.379707336425781 Time: 7.1255528926849365 s
 Result:
transported me beyond This ignorant present , and I feele , , and and , , , and and , ,
Epoch: 4/100, Batch: 129.0/128.5, Training loss: 6.214592933654785 Time: 7.133366584777832 s
 Result:
: Lay on Macduffe , And damn ' d be , , and and , , and , and , ,
Epoch: 5/100, Batch: 129.0/128.5, Training loss: 6.066814422607422 Time: 7.092589378356934 s
 Result:
the loyaltie I owe , In doing it , payes , , and and , , and , , and ,
Epoch: 6/100, Batch: 129.0/128.5, Training loss: 5.929080009460449 Time: 

In [0]:
bi_lstm_model = biLSTM(max_len,embedding_size,hidden_nodes,learning_rate)

In [0]:
bi_lstm_model.train(data_split_by_word,train_x,train_y,total_epoches,batch_size,10)

Epoch: 1/500, Batch: 129.0/128.5, Training loss: 7.314228057861328, Perplexity: 1501.5123291015625 Time: 45.96108078956604 s
 Result:
she speaks , I will set downe what comes from , , and to our , and , and to ,
Epoch: 2/500, Batch: 129.0/128.5, Training loss: 6.37598991394043, Perplexity: 587.5667724609375 Time: 47.18421769142151 s
 Result:
? Son . As Birds do Mother Wife . What , , and he , and , and , and ,
Epoch: 3/500, Batch: 129.0/128.5, Training loss: 5.715014457702637, Perplexity: 303.3885803222656 Time: 50.46971917152405 s
 Result:
Spirits , That tend on mortall thoughts , vnsex me to to . Exeunt . . Enter . . Enter .
Epoch: 4/500, Batch: 129.0/128.5, Training loss: 5.203893184661865, Perplexity: 181.97933959960938 Time: 52.58178639411926 s
 Result:
Drinke , Sir , is a great prouoker of three . . Exeunt . Scena . Enter . Enter . .
Epoch: 5/500, Batch: 129.0/128.5, Training loss: 4.787283897399902, Perplexity: 119.97505950927734 Time: 55.74652028083801 s
 Result:
that seemes to

KeyboardInterrupt: ignored